Setting SmugMug Print Size Keywords with Jupyter and Python
=========================================

![](jupysmug.png)

## Why am I doing this?

Many years ago I wrote a little J verb `smuprintsizes` that computed 
the [largest standard SmugMug print sizes](https://analyzethedatanotthedrivel.org/2010/02/21/assigning-smugmug-print-size-keys/) when given image dimensions 
and the desired [DPI](https://en.wikipedia.org/wiki/Dots_per_inch). I used 
the output of this verb to set aspect ratio keywords for my SmugMug pictures until changes to 
SmugMug, particularly the introduction of [OAuth authentication](https://en.wikipedia.org/wiki/OAuth), broke my little SmugMug API application that called `smugprintsizes`. 

My print size keyword setter broke years ago but many of these keys still show up in my "top ten" keywords. 

    10x15 4x5 4x6 5x5 5x6.7 5x7 ...
    
Print size keywords were very handy. They made it easy to select paper sizes for one or hundreds of pictures. 
This notebook will use the SmugMug API and Python to compute and set print size keywords.

## The Print Sizes Table

`smugprintsizes` made use of the following table.

     ┌─────┬─────────┬──────────────┐
     │0.7  │17.5 70  │3.5x5 7x10    │
     ├─────┼─────────┼──────────────┤
     │0.8  │20 80    │4x5 8x10      │
     ├─────┼─────────┼──────────────┤
     │0.755│21.2 84.8│4x5.3 8x10.6  │
     ├─────┼─────────┼──────────────┤
     │0.665│24 96    │4x6 8x12      │
     ├─────┼─────────┼──────────────┤
     │0.5  │32 50 128│4x8 5x10 8x16 │
     ├─────┼─────────┼──────────────┤
     │1    │25 64 100│5x5 8x8 10x10 │
     ├─────┼─────────┼──────────────┤
     │0.745│33.5     │5x6.7         │
     ├─────┼─────────┼──────────────┤
     │0.715│35       │5x7           │
     ├─────┼─────────┼──────────────┤
     │0.165│150      │5x30          │
     ├─────┼─────────┼──────────────┤
     │0.4  │160      │8x20          │
     ├─────┼─────────┼──────────────┤
     │0.775│93.5     │8.5x11        │
     ├─────┼─────────┼──────────────┤
     │0.75 │108      │9x12          │
     ├─────┼─────────┼──────────────┤
     │0.77 │130      │10x13         │
     └─────┴─────────┴──────────────┘
     
The first column is the Short/Long image aspect ratio rounded to 0.005. The middle column 
lists areas in square inches of the corresponding print sizes in the last column.

Finding the largest DPI dependent print size is simple matter of:

1. Divide the short image dimension by the long image dimension and round to 0.005.
   This is the aspect ratio.

2. Search for an aspect ratio match in the first column. Many images will not match.
   Quit and return `0z1` for no aspect match. The `0zN` codes are similiar to 
   the `NxM` print sizes codes. This will be important in later notebooks.

3. If a match is found compute the print area required for a given DPI and round to 0.5.

4. Find the index of the largest area in the second column that is greater than or equal to the required 
   area computed in the previous step. If there are not enough pixels no area will meet this criterion.
   Quit and return `0z0` for not enough pixels. 
   
5. If an area is found select and return the corresponding print size in the last column.



An image with dimensions of 2389 x 3344 has enough pixels to make a standard 5x7 inch 360 DPI print. 
It does not  have enough pixels to make a 5x7 inch 720 DPI print. 

Print resolution is a hot button issue for photographers. How many dots (DPI) or pixels (PPI) are 
required depends on many factors, viewing distance, illumination, image colors, paper gloss and so on. 
Human vision tests have demonstrated that young people with excellent eyesight can tell the difference
between 500 DPI and 600 DPI prints. Resolutions beyond 600 DPI are mostly wasted unless you are using loupes or microscopes.
[According to Dr. Optoglass](https://wolfcrow.com/blog/notes-by-dr-optoglass-the-resolution-of-the-human-eye/):

>*If the average reading distance is 1 foot (12 inches = 305 mm), p @0.4 arc minute is 35.5 microns or about 720 ppi/dpi. p @1 arc minute is 89 microns or about 300 dpi/ppi. This is why magazines are printed at 300 dpi – it’s good enough for most people. Fine art printers aim for 720, and that’s the best it need be. Very few people stick their heads closer than 1 foot away from a painting or photograph.*

Digital printers  complicate DPI issues by applying  sophisticated resizing algorithms that can turn low resolution 
originals into plausible higher resolution copies. I've found that 360 DPI is a good starting point for SmugMug prints.
For exceptional images you can simply divide the 360 DPI image dimensions by two for 720 DPI printing. 

## Computing DPI Dependent Print Area

The use of the print size table is clear with the exception of computing the print area required for a given DPI.
`dpi_area` computes DPI dependent print area.

In [29]:
def round_to(n, precision):
    correction = 0.5 if n >= 0 else -0.5
    return int( n/precision+correction ) * precision

def aspect_ratio(height, width, *, precision=0.005):
    return round_to( min(height, width) / max(height, width), precision )

def dpi_area(height, width, *, dpi=360, precision=0.5):
    return round_to( (height * width) / dpi ** 2, precision )

# image pixel dimensions - order is immaterial
height, width = 2389 , 3344

print('aspect ratio %s' % aspect_ratio(height, width))
print('area at 360 dpi %s' % dpi_area(height, width))
print('area at 720 dpi %s' % dpi_area(height, width, dpi=720))

aspect ratio 0.715
area at 360 dpi 61.5
area at 720 dpi 15.5


## Representing the Print Size table

There are many way to encode the print size table. I am starting with the simplest possible representation: three lists,
one for each column.

The lists must have the same number of items. Eventually, these details will be hidden within a `SmugPyter` subclass 
that manages the details of creating and using print size tables. For now let's use the following lists.

In [30]:
aspect_ratios = [0.7, 0.8, 0.755, 0.665, 0.5, 1, 0.745, 0.715, 0.165, 0.4, 0.775, 0.75, 0.77]
print(aspect_ratios)
print(len(aspect_ratios))

[0.7, 0.8, 0.755, 0.665, 0.5, 1, 0.745, 0.715, 0.165, 0.4, 0.775, 0.75, 0.77]
13


In [31]:
print_areas = [[17.5,70],[20,80],[21.2,84.8],[24,96],[32,50,128],[25,64,100],[33.5],[35],[150 ],[160],[93.5],[108 ],[130]]
print(print_areas)
print(len(print_areas))

[[17.5, 70], [20, 80], [21.2, 84.8], [24, 96], [32, 50, 128], [25, 64, 100], [33.5], [35], [150], [160], [93.5], [108], [130]]
13


## Minimum Print Size Area

Any image with a `dpi_area` below the minimum print size table area does not have enough pixels to print. 
It useful to know this value. The following `flatten` function 
from [Recipe 4.14, Python Cookbook 3rd Ed](https://www.safaribooksonline.com/library/view/python-cookbook-3rd/9781449357337/) makes it easy to extract this value.

In [32]:
from collections import Iterable

def flatten(items):
    """Yield items from any nested iterable; see REF."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x
            
min_print_area = min(list(flatten(print_areas)))
print(min_print_area)

17.5


In [33]:
size_keywords = [['3.5x5','7x10'],['4x5','8x10'],['4x5.3','8x10.6'],
                 ['4x6','8x12'],['4x8','5x10', '8x16'],['5x5','8x8','10x10'],['5x6.7'],
                 ['5x7'],['5x30'],['8x20'],['8.5x11'],['9x12'],['10x13']]
print(size_keywords)
print(len(size_keywords))

[['3.5x5', '7x10'], ['4x5', '8x10'], ['4x5.3', '8x10.6'], ['4x6', '8x12'], ['4x8', '5x10', '8x16'], ['5x5', '8x8', '10x10'], ['5x6.7'], ['5x7'], ['5x30'], ['8x20'], ['8.5x11'], ['9x12'], ['10x13']]
13


In [34]:
def print_size_key(height, width, *, no_ratio='0z1', no_pixels='0z0', min_area=17.5, ppi=360):
    """
    Compute print size key word from image dimensions. The result is a character string.
    
      key360 = print_size_key(2000, 3000)
      key720 = print_size_key(2000, 3000, ppi=720) # (ppi) is identical to dpi here
    """
    
    # basic argument check
    error_message = '(height), (width) must be positive integers'
    if not (isinstance(height, int) and isinstance(width, int)):
        raise ValueError(error_message)
    elif height <= 0 or width <= 0:
        raise ValueError(error_message)
    
    # area must exceed a minimum size
    print_area = dpi_area(height, width, dpi=ppi)
    if print_area < min_area:
        return no_pixels
    
    print_ratio = aspect_ratio(height, width)
    print_key = no_ratio
    for i, ratio in enumerate(aspect_ratios):
        if abs(print_ratio - ratio) <= 0.000005:
            print_key = no_pixels
            
            # not enough or more than enough area
            if print_area < print_areas[i][0]:
                break
            elif print_area > print_areas[i][-1]:
                print_key = size_keywords[i][-1]
                break     
            
            for j, area in enumerate(print_areas[i]):
                if area >= print_area and 0 < j:
                    print_key = size_keywords[i][j - 1]
                    break
                    
    return print_key
    
# many sizes available for aspect ratio 1.0
print('3800x3800 at 360 DPI = %s' % print_size_key(3800, 3800))
print('3800x3800 at 720 DPI = %s' % print_size_key(3800, 3800, ppi=720))
print('3000x3000 at 360 DPI = %s' % print_size_key(3000, 3000))
print('2000x2000 at 360 DPI = %s' % print_size_key(2000, 2000))

# not enough pixels
print('500x500 at 360 DPI = %s' % print_size_key(500,500))
print('10x10 at 360 DPI = %s' % print_size_key(10,10)) 

# no ratio 
print('3255x4119 at 360 DPI = %s' % print_size_key(3255, 4119))

3800x3800 at 360 DPI = 10x10
3800x3800 at 720 DPI = 5x5
3000x3000 at 360 DPI = 8x8
2000x2000 at 360 DPI = 5x5
500x500 at 360 DPI = 0z0
10x10 at 360 DPI = 0z0
3255x4119 at 360 DPI = 0z1


## Testing `print_size_key`

The `print_size_key` function seems simple enough but when I see three `break` statements in a loop
I set my bullshit detector to 
[eleven](https://duckduckgo.com/?q=you+tube+loudness+to+eleven&ia=videos&iax=videos&iai=4xgx4k83zzc) and start 
looking for bugs.

In [35]:
print_size_key('not', 'even_wrong') # throw exception

ValueError: (height), (width) must be positive integers

In [36]:
print_size_key(-2, -3) # throw exception

ValueError: (height), (width) must be positive integers

In [37]:
print_size_key(0, 50) # throw exception

ValueError: (height), (width) must be positive integers

In [38]:
print('0z0' == print_size_key(1,1))      # not enough pixels
print('0z0' == print_size_key(20,20))    # not enough pixels
print('0z0' == print_size_key(500,500))  # not enough pixels

True
True
True


In [39]:
print('0z1' == print_size_key(2000,2100))  # ratio not in table
print('0z1' == print_size_key(4000,3500))  # ratio not in table
print('0z1' == print_size_key(1000,5000))  # ratio not in table

True
True
True


As `print_size_key` rounds ratios and areas you need slightly more pixels than you might expect 
for a given print size. In practice this is not an issue as digital images usually have
more than enough pixels for small standard size prints.

In [41]:
print('0z0' == print_size_key(int(3.5 * 350), 5 * 350))           # 3.5x5 not enough pixels
print('3.5x5' == print_size_key(int(3.5 * 362), 5 * 362))         # 3.5x5
print('7x10' == print_size_key(7 * 362, 10 * 362))                # 7x10
print('5x6.7' == print_size_key(5 * 362, int(6.7 * 362)))         # 5x6.7
print('8.5x11' == print_size_key(int(8.5 * 362), 11 * 362))       # 8.5x11
print('10x10' == print_size_key(10 * 362, 10 * 362))              # 10x10
print('10x10' == print_size_key(10 * 722, 10 * 722, ppi=720))     # 10x10 at 720 DPI
print('5x30' == print_size_key(5 * 362, 30 * 362))                # 5x30
print('5x10' == print_size_key(5 * 722, 10 * 722, ppi=720))       # 5x10 at 720 DPI

True
True
True
True
True
True
True
True
True


## Calculating Print Size Keys for SmugMug Album Manifest Files

In the first notebook of this series I used the SmugMug API SmugMug to generate folders and files
containing SmugMug image metadata stored in CSV TAB delimited files. Now I will read these manifest files and
compute print size keys.

## Next on the Agenda!

Remember, always [Analyze the Data not the Drivel](https://analyzethedatanotthedrivel.org/).

John Baker, Meridian Idaho